# Test of the clasification pipeline

In [1]:
import os

# Function to change to the parent directory
def change_to_parent_directory():
    # Check if the directory has already been changed
    if not os.environ.get('DIR_CHANGED'):
        try:
            current_dir = os.path.dirname(os.path.abspath(__file__))
        except NameError:
            current_dir = os.getcwd()
        parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
        os.chdir(parent_dir)
        os.environ['DIR_CHANGED'] = '1'
        print(f"Current working directory changed to: {os.getcwd()}")
    else:
        print("Directory has already been changed.")

# Call the function to change the working directory
change_to_parent_directory()

Current working directory changed to: /home/ihranicky/git/domainradar-clf


## Optional: Create testing dataset
Note: If you want do to this, set create_test_parquet to **True**

In [2]:
create_test_parquet = False

if create_test_parquet:
    import pandas as pd
    import numpy as np

    # List of input Parquet files along with their maximum rows and desired labels
    input_files = [
        {'file': 'testdata/2405_clftest_benign_filtered.parquet', 'max_rows': 2000, 'label': 'benign'},
        {'file': 'testdata/2405_clftest_phishing_filtered.parquet', 'max_rows': 480, 'label': 'phishing'},
        {'file': 'testdata/2405_clftest_malware_filtered.parquet', 'max_rows': 292, 'label': 'malware'},
        {'file': 'testdata/dga_2310.parquet', 'max_rows': 300, 'label': 'dga'},
    ]

    # Number of rows to select in total
    n_rows = 3072
    
    # Read the first file to get the initial columns and create the first dataframe
    first_file_info = input_files[0]
    combined_df = pd.read_parquet(first_file_info['file'])
    
    # Limit the number of rows if necessary for the first file
    if len(combined_df) > first_file_info['max_rows']:
        combined_df = combined_df.sample(n=first_file_info['max_rows'], random_state=1)
    
    # Overwrite the "label" column with the specified label for the first file
    combined_df['label'] = first_file_info['label']
    
    # Get the columns from the first dataframe
    all_columns = combined_df.columns.tolist()

    # Process the remaining files
    for file_info in input_files[1:]:
        df = pd.read_parquet(file_info['file'])
        
        # Limit the number of rows if necessary
        if len(df) > file_info['max_rows']:
            df = df.sample(n=file_info['max_rows'], random_state=1)
        
        # Overwrite the "label" column with the specified label
        df['label'] = file_info['label']
        
        # Ensure all columns from the first dataframe are present
        for col in all_columns:
            if col not in df.columns:
                df[col] = None
        
        # Align the dataframe to the columns of the first dataframe
        df = df[all_columns]
        
        # Append the dataframe to the combined dataframe
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # Randomly select n_rows rows from the combined DataFrame
    selected_rows = combined_df.sample(n=n_rows, random_state=1)  # random_state for reproducibility
    
    # Save the selected rows to a new Parquet file
    selected_rows.to_parquet('testdata/validation.parquet')

In [3]:
!pip3 install matplotlib

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## Run classification

In [4]:
# Specify the parquet file with the dataset for classification
test_dataset = 'testdata/validation.parquet'

# Number of domain names to classify with each run of the pipeline (0 = classify all)
CHUNK_SIZE = 30

In [ ]:
import pandas as pd
import numpy as np
import shap
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from classifiers.pipeline import Pipeline
from classifiers.options import PipelineOptions
import matplotlib.pyplot as plt

# Initialize the classification pipeline
clf_options = PipelineOptions()
clf = Pipeline(clf_options)

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

# Function to map labels to 'benign' or 'malign'
def map_label(label):
    if label == 'benign':
        return 'benign'
    else:
        return f'malign ({label})'

# Function to convert labels to binary classes
def binary_label(label):
    return 'negative' if label == 'benign' else 'positive'

# Apply label mapping
input_df['mapped_label'] = input_df['label'].apply(map_label)
input_df['binary_label'] = input_df['label'].apply(binary_label)

# Ensure SHAP JavaScript initialization
shap.initjs()

# Determine the number of chunks
CHUNK_SIZE = 1000  # Define CHUNK_SIZE
num_chunks = (len(input_df) + CHUNK_SIZE - 1) // CHUNK_SIZE if CHUNK_SIZE > 0 else 1

# Initialize counters for overall statistics
total_true_labels = []
total_pred_labels = []

# Format string for aligned output
header_format_str = "{:<3} | {:<50} | {:<18} | {:<10} | {:<10}"
data_format_str = "{:<3} | {:<50} | {:<18} | {:<10} | {:.6f}"

# Process the dataframe in chunks
for i in range(num_chunks):
    if CHUNK_SIZE > 0:
        start_idx = i * CHUNK_SIZE
        end_idx = start_idx + CHUNK_SIZE
        chunk_df = input_df[start_idx:end_idx]
    else:
        chunk_df = input_df

    # Perform your classification or processing on the working_df here
    print(f"===== Processing chunk {i+1}/{num_chunks} =====")

    chunk_without_label = chunk_df.drop(columns=['label', 'mapped_label', 'binary_label']) # Label should not be known to classifiers
    chunk_results = clf.classify_domains(chunk_without_label)

    # Collect predictions and true labels
    true_labels = chunk_df['binary_label'].values
    pred_labels = []
    for result in chunk_results:
        pred_label = 'negative' if result['aggregate_probability'] < 0.5 else 'positive'
        pred_labels.append(pred_label)

    # Update overall statistics
    total_true_labels.extend(true_labels)
    total_pred_labels.extend(pred_labels)

    # Display header for results
    print(header_format_str.format("Res", "Domain Name", "Actual Label", "Predicted", "Probability"))

    # Display results for each domain
    for idx, result in enumerate(chunk_results):
        actual_label = chunk_df.iloc[idx]['mapped_label']
        predicted_label = 'benign' if pred_labels[idx] == 'negative' else 'malign'
        domain_name = result['domain_name']
        aggregate_probability = result['aggregate_probability']
        status = "OK" if pred_labels[idx] == true_labels[idx] else "ER"
        print(data_format_str.format(status, domain_name[:50], actual_label, predicted_label, aggregate_probability))
        
        # Run debug_domain method for misclassified domains
        if status == "ER":
            print(f"Debugging misclassified domain: {domain_name}")
            debug_data = clf.debug_domain(domain_name, chunk_df, n_top_features=10)

            # Print out the top n feature importances and values for each classifier
            for classifier, data in debug_data.items():
                print(f"\nClassifier: {classifier}")
                print(f"Top {len(data['top_features'])} features for domain '{domain_name}':")
                for feature_info in data['top_features']:
                    print(f"Feature: {feature_info['feature']}, Value: {feature_info['value']}, SHAP Value: {feature_info['shap_value']}")
                
                # Display the force plot for phishing_lgbm
                if classifier == "phishing_cnn":
                    base_value, shap_values, domain_row = data['force_plot_data']
                    shap.force_plot(base_value, shap_values, domain_row)
                    plt.show()

    # Calculate metrics for the current chunk
    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels, pos_label='positive', average='binary')
    recall = recall_score(true_labels, pred_labels, pos_label='positive', average='binary')
    f1 = f1_score(true_labels, pred_labels, pos_label='positive', average='binary')

    tn, fp, fn, tp = confusion_matrix(true_labels, pred_labels, labels=['negative', 'positive']).ravel()
    false_positives = fp
    false_negatives = fn
    total_positives = tp + fp
    total_negatives = tn + fn

    fp_ratio = (false_positives / total_positives) if total_positives > 0 else 0
    fn_ratio = (false_negatives / total_negatives) if total_negatives > 0 else 0

    print(f"Chunk {i+1}/{num_chunks} metrics:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"False Positives: {false_positives} ({fp_ratio * 100:.2f}%)")
    print(f"False Negatives: {false_negatives} ({fn_ratio * 100:.2f}%)")
    print(f"===== Chunk {i+1}/{num_chunks} completed. =====")

# Calculate overall metrics
overall_accuracy = accuracy_score(total_true_labels, total_pred_labels)
overall_precision = precision_score(total_true_labels, total_pred_labels, pos_label='positive', average='binary')
overall_recall = recall_score(total_true_labels, total_pred_labels, pos_label='positive', average='binary')
overall_f1 = f1_score(total_true_labels, total_pred_labels, pos_label='positive', average='binary')

overall_tn, overall_fp, overall_fn, overall_tp = confusion_matrix(total_true_labels, total_pred_labels, labels=['negative', 'positive']).ravel()
overall_false_positives = overall_fp
overall_false_negatives = overall_fn
overall_total_positives = overall_tp + overall_fp
overall_total_negatives = overall_tn + overall_fn

overall_fp_ratio = (overall_false_positives / overall_total_positives) if overall_total_positives > 0 else 0
overall_fn_ratio = (overall_false_negatives / overall_total_negatives) if overall_total_negatives > 0 else 0

print("Overall metrics:")
print(f"Overall Accuracy: {overall_accuracy}")
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1 Score: {overall_f1}")
print(f"Overall False Positives: {overall_false_positives} ({overall_fp_ratio * 100:.2f}%)")
print(f"Overall False Negatives: {overall_false_negatives} ({overall_fn_ratio * 100:.2f}%)")


/home/ihranicky/git/domainradar-clf/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-10 16:42:51.986474: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 16:42:51.986497: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 16:42:51.987327: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-10 16:42:51.992222: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Ten

===== Processing chunk 1/4 =====


In [4]:
import pandas as pd
import numpy as np
from classifiers.pipeline import Pipeline

# Initialize the classification pipeline
clf = Pipeline()

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

input_df = input_df.sample(frac=1).reset_index(drop=True)

#preliminary_results_df = clf.generate_preliminary_results(input_df, output_file="test_preliminary_results.parquet")
preliminary_results_df = clf.generate_preliminary_results(input_df, add_final=True)

2024-06-05 15:43:44.790733: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 15:43:44.790757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 15:43:44.791508: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-05 15:43:44.795713: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 15:43:45.449294: W tensorflow/compiler/tf2

In [5]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = '{:.6f}'.format
preliminary_results_df

,domain_name,dns_available,dns_nonzero,tls_available,tls_nonzero,ip_available,ip_nonzero,rdap_available,rdap_nonzero,geo_available,geo_nonzero,label,phishing_cnn_result,phishing_lgbm_result,malware_cnn_result,malware_xgboost_result,dga_binary_nn_result,phishing_sum,phishing_avg,phishing_prod,malware_sum,malware_avg,malware_prod,total_sum,total_avg,total_prod,badness_probability,phishing_xgboost_result,phishing_deepnn_result
0,cag.chessly.com,0.600000,0.250000,1.000000,0.750000,1.000000,0.625000,1.000000,0.833333,1.000000,0.722222,benign,-43043.625000,0.992818,-42264.335938,0.744956,0.000002,-43042.632182,-21521.316091,-42734.498727,-42263.590982,-21131.795491,-31485.068823,-85306.223162,-17061.244632,2252.117563,0.950000,0.999854,0.079653
1,michaelavancatova.cz,1.000000,0.700000,1.000000,0.833333,1.000000,0.625000,0.958333,0.541667,1.000000,0.666667,benign,37216.121094,0.986598,47110.347656,0.993814,0.046707,37217.107692,18608.553846,36717.346046,47111.341470,23555.670735,46818.905799,84328.495869,16865.699174,80293036.254983,0.950000,0.425042,0.004687
2,t.shareearn.net,0.600000,0.200000,1.000000,0.833333,1.000000,0.625000,1.000000,0.708333,1.000000,0.666667,benign,10205.281250,0.999980,-7465.998047,0.935921,0.000048,10206.281230,5103.140615,10205.077654,-7465.062125,-3732.531063,-6987.588018,2741.219153,548.243831,-3453.930243,0.950000,0.999765,0.637165
3,download.savannah.nongnu.org,0.900000,0.525000,1.000000,0.750000,1.000000,0.625000,0.958333,0.791667,1.000000,0.722222,benign,11162.598633,0.991105,-6793.439453,0.802605,0.000000,11163.589738,5581.794869,11063.308533,-6792.636849,-3396.318424,-5452.445457,4370.952889,874.190578,-0.000000,0.950000,0.996077,0.711941
4,zonefiles.io,1.000000,0.750000,1.000000,0.833333,1.000000,0.750000,1.000000,0.625000,1.000000,1.000000,benign,19649.228516,0.999927,-5083.651367,0.999998,0.008958,19650.228442,9825.114221,19647.788399,-5082.651369,-2541.325685,-5083.639853,14567.586031,2913.517206,-894746.557522,0.950000,0.999911,0.999126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3067,gaynaletube.com,0.900000,0.525000,1.000000,0.458333,1.000000,0.625000,1.000000,0.583333,1.000000,0.722222,benign,6979.881836,0.991566,-18564.382812,0.999997,0.102782,6980.873402,3490.436701,6921.012718,-18563.382815,-9281.691408,-18564.334125,-11582.406631,-2316.481326,-13205895.793595,0.950000,0.998471,0.751355
3068,r.honeygain.me,0.600000,0.250000,1.000000,0.833333,1.000000,0.750000,0.958333,0.708333,1.000000,1.000000,benign,-3467.795166,0.999318,-25042.314453,0.666322,0.000001,-3466.795848,-1733.397924,-3465.431529,-25041.648131,-12520.824066,-16686.237409,-28508.443978,-5701.688796,61.515191,0.950000,0.998359,1.000000
3069,vidmate.network,0.950000,0.650000,1.000000,0.833333,1.000000,0.750000,1.000000,0.750000,1.000000,1.000000,benign,16276.885742,0.999998,-12013.079102,0.999993,0.000045,16277.885740,8138.942870,16276.854981,-12012.079109,-6006.039554,-12012.990313,4265.806677,853.161335,-8830.780486,0.950000,0.999995,0.999844
3070,dexa.ai,1.000000,0.750000,1.000000,0.791667,1.000000,0.625000,0.833333,0.750000,1.000000,0.722222,benign,2241.506592,0.995999,-22908.941406,0.999995,0.129624,2242.502591,1121.251296,2232.538792,-22907.941411,-11453.970706,-22908.821244,-20665.309196,-4133.061839,-6629609.616726,0.950000,0.995980,1.000000


In [6]:
preliminary_results_df[["domain_name", "label", "badness_probability", "phishing_cnn_result", "phishing_deepnn_result", "phishing_lgbm_result", "phishing_xgboost_result", "malware_cnn_result", "malware_xgboost_result", "dga_binary_nn_result"]]

,domain_name,label,badness_probability,phishing_cnn_result,phishing_deepnn_result,phishing_lgbm_result,phishing_xgboost_result,malware_cnn_result,malware_xgboost_result,dga_binary_nn_result
0,cag.chessly.com,benign,0.950000,-43043.625000,0.079653,0.992818,0.999854,-42264.335938,0.744956,0.000002
1,michaelavancatova.cz,benign,0.950000,37216.121094,0.004687,0.986598,0.425042,47110.347656,0.993814,0.046707
2,t.shareearn.net,benign,0.950000,10205.281250,0.637165,0.999980,0.999765,-7465.998047,0.935921,0.000048
3,download.savannah.nongnu.org,benign,0.950000,11162.598633,0.711941,0.991105,0.996077,-6793.439453,0.802605,0.000000
4,zonefiles.io,benign,0.950000,19649.228516,0.999126,0.999927,0.999911,-5083.651367,0.999998,0.008958
...,...,...,...,...,...,...,...,...,...,...
3067,gaynaletube.com,benign,0.950000,6979.881836,0.751355,0.991566,0.998471,-18564.382812,0.999997,0.102782
3068,r.honeygain.me,benign,0.950000,-3467.795166,1.000000,0.999318,0.998359,-25042.314453,0.666322,0.000001
3069,vidmate.network,benign,0.950000,16276.885742,0.999844,0.999998,0.999995,-12013.079102,0.999993,0.000045
3070,dexa.ai,benign,0.950000,2241.506592,1.000000,0.995999,0.995980,-22908.941406,0.999995,0.129624


In [20]:
preliminary_results_df[["domain_name", "label", "badness_probability"]]

,domain_name,label,badness_probability
0,est1.hlsvideocdn78.shop,benign,1.000000
1,hp18.pages.dev,phishing,0.999990
2,www.charlotteobserver.com,benign,0.000260
3,zakazky.pardubickykraj.cz,benign,0.000168
4,vid-s16-n10-nl-cdn.eporner.com,benign,0.000192
5,zgktpslfgyflqi.com,dga,1.000000
6,www.abanca.com,benign,0.000777
7,reviewholdreviewho.wixsite.com,phishing,0.999996
8,covers.alibrate.com,benign,0.999995
9,lfjvcjow.org,dga,0.999999
